## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,bengkulu,100,ID,overcast clouds,88,-3.8004,102.2655,75.74,0.0,0.0,3.02
1,1,barrow,90,US,haze,77,71.2906,-156.7887,-3.98,0.0,0.0,3.44
2,2,knin,97,HR,overcast clouds,93,44.0439,16.2000,45.59,0.0,0.0,6.40
3,3,pacific grove,1,US,clear sky,68,36.6177,-121.9166,56.88,0.0,0.0,5.01
4,4,yenagoa,33,NG,scattered clouds,89,4.9247,6.2642,78.78,0.0,0.0,4.25


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation?"))
max_temp = float(input("What is the maximum temperature you would like for your vacation?"))

What is the minimum temperature you would like for your vacation?75
What is the maximum temperature you would like for your vacation?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filtered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                      (city_data_df["Max Temp"] >= min_temp)]
filtered_cities_df.head(10)

,City_ID,City,Cloudiness,Country,Current Description,Humidity,Lat,Lng,Max Temp,Rain (inches),Snow (inches),Wind Speed
0,0,bengkulu,100,ID,overcast clouds,88,-3.8004,102.2655,75.74,0.0,0.0,3.02
4,4,yenagoa,33,NG,scattered clouds,89,4.9247,6.2642,78.78,0.0,0.0,4.25
7,7,east london,96,ZA,overcast clouds,54,-33.0153,27.9116,79.02,0.0,0.0,11.65
10,10,hithadhoo,99,MV,overcast clouds,75,-0.6000,73.0833,81.21,0.0,0.0,8.66
11,11,cayenne,0,GF,clear sky,90,4.9333,-52.3333,79.09,0.0,0.0,2.30
13,13,carnarvon,0,AU,clear sky,60,-24.8667,113.6333,75.27,0.0,0.0,21.85
14,14,ternate,78,ID,broken clouds,81,0.8000,127.4000,82.74,0.0,0.0,6.58
20,20,rikitea,67,PF,broken clouds,69,-23.1203,-134.9692,76.33,0.0,0.0,18.14
26,26,ocos,4,GT,clear sky,75,14.5094,-92.1933,86.50,0.0,0.0,5.61
27,27,bodden town,20,KY,few clouds,83,19.2833,-81.2500,80.60,0.0,0.0,3.00


In [5]:
# 4a. Determine if there are any empty rows.
filtered_cities_df.count()

City_ID                161
City                   161
Cloudiness             161
Country                159
Current Description    161
Humidity               161
Lat                    161
Lng                    161
Max Temp               161
Rain (inches)          161
Snow (inches)          161
Wind Speed             161
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filtered_cities_df.dropna()
clean_df.count()

City_ID                159
City                   159
Cloudiness             159
Country                159
Current Description    159
Humidity               159
Lat                    159
Lng                    159
Max Temp               159
Rain (inches)          159
Snow (inches)          159
Wind Speed             159
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,bengkulu,ID,75.74,overcast clouds,-3.8004,102.2655,
4,yenagoa,NG,78.78,scattered clouds,4.9247,6.2642,
7,east london,ZA,79.02,overcast clouds,-33.0153,27.9116,
10,hithadhoo,MV,81.21,overcast clouds,-0.6000,73.0833,
11,cayenne,GF,79.09,clear sky,4.9333,-52.3333,
13,carnarvon,AU,75.27,clear sky,-24.8667,113.6333,
14,ternate,ID,82.74,broken clouds,0.8000,127.4000,
20,rikitea,PF,76.33,broken clouds,-23.1203,-134.9692,
26,ocos,GT,86.50,clear sky,14.5094,-92.1933,
27,bodden town,KY,80.60,few clouds,19.2833,-81.2500,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f'{lat},{lng}'

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
        
    except (IndexError): 
        print("Hotel not found... skipping.")

hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,bengkulu,ID,75.74,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
4,yenagoa,NG,78.78,scattered clouds,4.9247,6.2642,Aridolf Resort Wellness and Spa
7,east london,ZA,79.02,overcast clouds,-33.0153,27.9116,Tu Casa
10,hithadhoo,MV,81.21,overcast clouds,-0.6000,73.0833,Scoop Guest House
11,cayenne,GF,79.09,clear sky,4.9333,-52.3333,Hôtel Le Dronmi
13,carnarvon,AU,75.27,clear sky,-24.8667,113.6333,Hospitality Carnarvon
14,ternate,ID,82.74,broken clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
20,rikitea,PF,76.33,broken clouds,-23.1203,-134.9692,People ThankYou
26,ocos,GT,86.50,clear sky,14.5094,-92.1933,Auto Hotel El Soñador
27,bodden town,KY,80.60,few clouds,19.2833,-81.2500,Will T Place


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count(), hotel_df.notnull().sum(), hotel_df.isnull().sum()

(City                   159
 Country                159
 Max Temp               159
 Current Description    159
 Lat                    159
 Lng                    159
 Hotel Name             159
 dtype: int64,
 City                   159
 Country                159
 Max Temp               159
 Current Description    159
 Lat                    159
 Lng                    159
 Hotel Name             159
 dtype: int64,
 City                   0
 Country                0
 Max Temp               0
 Current Description    0
 Lat                    0
 Lng                    0
 Hotel Name             0
 dtype: int64)

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd><dd>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))